In [113]:
"""
ATFM
@author: Adrian Sanz Mena
"""
# pyright: reportWildcardImportFromLibrary=false

from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
from coopr.pyomo import *

# Function Definition

Given Functions

In [114]:
def D_k_ini(D_k, time_intervals):
    for i in range(1, time_intervals+1):
        D_k["A",i] = 1
        D_k["B",i] = 1
        D_k["C",i] = 1
    return D_k

def A_k_ini(A_k, time_intervals):
    for i in range(1, time_intervals+1):
        A_k["A",i] = 1
        A_k["B",i] = 1
        A_k["C",i] = 1
    return A_k

def S_j_ini(S_j, time_intervals):
    for i in range(1, time_intervals+1):
        S_j["I",i]   = 4
        S_j["II",i]  = 4
        S_j["III",i] = 4
        S_j["IV",i]  = 4
    return S_j

def arcs_ini(arcs):
    arcs["f1"] = (  "(A,q(A))","(q(A),a)","(a,b)","(b,C)"  )
    arcs["f2"] = (  "(A,q(A))","(q(A),e)","(e,B)"  )
    arcs["f3"] = (  "(B,q(B))","(q(B),f)","(f,C)"  )
    return arcs

def N_f_ini(N_f):
    N_f["f1"] = ("A","q(A)","a","b","C")
    N_f["f2"] = ("A","q(A)","e","B")
    N_f["f3"] = ("B","q(B)","f","C")
    return N_f

def l_fnm_ini(l_fnm):
    l_fnm["f1", "(A,q(A))"] = 0
    l_fnm["f1", "(q(A),a)"] = 5
    l_fnm["f1", "(a,b)"]    = 8
    l_fnm["f1", "(b,C)"]    = 1
    l_fnm["f2", "(A,q(A))"] = 0
    l_fnm["f2", "(q(A),e)"] = 5
    l_fnm["f2", "(e,B)"]    = 7
    l_fnm["f3", "(B,q(B))"] = 0
    l_fnm["f3", "(q(B),f)"] = 3
    l_fnm["f3", "(f,C)"]    = 3
    return l_fnm

def r_fn_ini(r_fn, model):
    for k in model.F:
        for i1, val1 in enumerate(model.N_f[k]):
            r_fn[k, val1] = model.d_f[k]
            for i2, val2 in enumerate(model.arcs[k]):
                if i2<i1:
                    r_fn[k, val1] = r_fn[k, val1] + model.l_fnm[k, val2]
    return r_fn

def G_ini(G):
    G["f1"] = 2
    G["f2"] = 3
    G["f3"] = 3
    return G

def A_ini(A):
    A["f1"] = 5
    A["f2"] = 4
    A["f3"] = 4
    return A

def T_fn_ini(T_fn, model, time_intervals):
    for k in model.F:
        for i,val in enumerate(model.N_f[k]):
            if i<2:
                T_fn[k,val] = np.linspace(model.r_fn[k, val], min(model.r_fn[k,val] + model.G[k], time_intervals), min(model.r_fn[k,val]+model.G[k],time_intervals)-model.r_fn[k,val]+1)
            if i>=2:
                T_fn[k, val] = np.linspace(model.r_fn[k, val], min(model.r_fn[k,val]+model.G[k]+model.A[k],time_intervals), min(model.r_fn[k,val]+model.G[k]+model.A[k],time_intervals)-model.r_fn[k,val]+1)
    return T_fn

def ca_ini(ca):
    ca["f1"] = 3
    ca["f2"] = 2.5
    ca["f3"] = 2.5
    return ca

def cg_ini(cg):
    cg["f1"] = 1.5
    cg["f2"] = 1.5
    cg["f3"] = 1.5
    return cg

def N_S_ini(N_S):
    N_S["f1", "I"]   = "A", "q(A)", "a"
    N_S["f1", "II"]  = "a", "b"
    N_S["f1", "III"] = ""
    N_S["f1", "IV"]  = "b", "C"

    N_S["f2", "I"]   = "A", "q(A)", "e"
    N_S["f2", "II"]  = ""
    N_S["f2", "III"] = "e", "B"
    N_S["f2", "IV"]  = ""
    
    N_S["f3", "I"]   = ""
    N_S["f3", "II"]  = ""
    N_S["f3", "III"] = "B", "q(B)", "f"
    N_S["f3", "IV"]  = "f", "C"
    return N_S

def arc_S_ini(arc_S):
    arc_S["f1", "I"]    = "(A,q(A))", "(q(A),a)"
    arc_S["f1", "II"]   = "(a,b)"
    arc_S["f1", "III"]  = {}
    arc_S["f1", "IV"]   = "(b,C)"

    arc_S["f2", "I"]    = "(A,q(A))", "(q(A),e)"
    arc_S["f2", "II"]   = ""
    arc_S["f2", "III"]  = "(e,B)"
    arc_S["f2", "IV"]   = ""
    
    arc_S["f3", "I"]    = ""
    arc_S["f3", "II"]   = ""
    arc_S["f3", "III"]  = "(B,q(B))", "(q(B),f)"
    arc_S["f3", "IV"]   = "(f,C)"
    return arc_S


Self-Made Functions

In [ ]:
def Objective_rule(model):
    Total_Obj = 0.0

    for _,f in enumerate(model.F):
        Obj_f = 0.0
        
    # for orig in model.origin_f[f]:
        orig = model.origin_f[f]
        for t in model.T_fn[f, orig]:
            if t==0:
                continue
            Obj_f += (model.cg[f] - model.ca[f]) * (model.w[f, orig, t] - model.w[f, orig, t-1]) * (t - model.d_f[f])

    # for dest in model.destination_f[f]:
        dest = model.destination_f[f]
        for t in model.T_fn[f, dest]:
            if t==0:
                continue
            Obj_f +=                 model.ca[f] * (model.w[f, dest, t] - model.w[f, dest, t-1]) * (t - model.a_f[f])
        
        Total_Obj += Obj_f

    return Total_Obj

def departure_capacity_constraint(model, k, t):
    if t == 0:
        return Constraint.Skip #This is to skip the constratin at t=0.
    
    # Check if any flight has origin 'k'
    has_flights_from_k = any(model.origin_f[f] == k for f in model.F)
    
    #Complete the rest of the instructions for the constraint.
    if not has_flights_from_k: # If there is no flight with origin 'k' skip the constraint (JIC)
        return Constraint.Skip

    return sum(model.w[f, k, t] - model.w[f, k, t-1] for f in model.F if k == model.origin_f[f]) <= model.D_k[k, t]

def arrival_capacity_constraint(model, k, t):
    if t == 0:
        return Constraint.Skip #This is to skip the constratin at t=0.
    
    # Check if any flight has origin 'k'
    receives_flights_from_k = any(model.destination_f[f] == k for f in model.F)
    
    #Complete the rest of the instructions for the constraint.
    if not receives_flights_from_k: # If there is no flight with desintation 'k' skip the constraint (JIC)
        return Constraint.Skip

    return sum(model.w[f, k, t] - model.w[f, k, t-1] for f in model.F if k == model.destination_f[f]) <= model.A_k[k, t]

def sector_capacity_constraint(model, j, t):
    if t == 0:
        return Constraint.Skip #This is to skip the constratin at t=0.
    
    # Check which flights cross sector j
    flights_in_j = [f for f in model.F if (f, j) in model.arc_S]

    if not flights_in_j: # If no flights cross sector j (JIC)
        return Constraint.Skip

    sec_constr = 0
    for f in flights_in_j:
        nodes = list(model.N_S[f, j])  # convertir a lista para poder usar índices

        for idx, n in enumerate(nodes):
            if idx < len(nodes)-1:

                n_next = nodes[idx + 1]
                sec_constr += model.w[f, n, t] - model.w[f, n_next, t]

            else:
                sec_constr += model.w[f, n, t]

    return sec_constr <= model.S_j[j, t]

def flight_structure_constraints_1_rule(model, f, node, t):
    if t==0:
        return Constraint.Skip
    
    n = list(model.N_f[f]).index(node)

    if n < len(model.N_f[f]) - 1:
        m = n+1
        m_node = list(model.N_f[f])[m]

        arc = f"({node},{m_node})"
        l = model.l_fnm[f, arc]      
            
        return model.w[f, m_node, int(t + l)] - model.w[f, node, int(t)] <= 0
    else:
        return Constraint.Skip
 
def flight_structure_constraints_2_rule(model, f, _, t):
    orig, dest = model.origin_f[f], model.destination_f[f]

    t_dest = int(t + model.a_f[f] - model.d_f[f] + model.A[f])

    if t_dest not in model.T:
        return Constraint.Skip

    return model.w[f, orig, t] - model.w[f, dest, t_dest] <= 0

def flight_structure_constraints_3_rule(model, f, _, t):
    # What if we had C = [f1, f2, f3....] ?
    if f not in model.C:
        return Constraint.Skip

    #  If there is any f (1st flight) in model F, whose destination is the origin of another f_
    flights_with_cont = [f_ for f_ in model.F 
                         if model.origin_f[f_] == model.destination_f[f]]
    
    # If there are not continued flights skip it
    if not flights_with_cont:
        return Constraint.Skip
    
    # The flight that is continuing (2nd)
    f_ = flights_with_cont[0]  
    
    t_dest = int(t - model.s_f[f])
    
    if t_dest not in model.T:
        return Constraint.Skip
    
    return model.w[f_, model.origin_f[f_], t] - model.w[f, model.destination_f[f], t_dest] <= 0

def flight_structure_constraints_4_rule(model, f, node, t):
    t_global_end = model.T.last()

    t_node_start = model.T_fn[f, node].first()
    
    # Avoid indexing an imposible time for a node 
    if t <= t_node_start: 
        return Constraint.Skip
    
    if t > t_global_end:
        return Constraint.Skip
    else:    
        return model.w[f, node, t-1] - model.w[f, node, t] <= 0

def flight_structure_constraints_5_rule(model, f, node):
    T_fn_sup = model.T_fn[f, node].last()
    
    if T_fn_sup in model.T:
        return model.w[f, node, T_fn_sup] == 1
    
def flight_structure_constraints_6_rule(model, f, node, t):
    T_fn_sub = model.T_fn[f, node].first()
    
    if t <= T_fn_sub - 1:
        return model.w[f, node, t] == 0
    else:
        return Constraint.Skip


# Initialize Variables and Dicts.

In [116]:
#Dimensions
n_airports = 3
n_sectors = 4
n_flights = 3
time_intervals = 40

#Sets
T = list(range(time_intervals+1))
F = ['f1', 'f2', 'f3']
K = ['A', 'B', 'C']
S = ['I', 'II', 'III', 'IV']
C = [F[1], F[2]]

# Departure capacity of airport K at time t
D_k = {}

# Arrival capacity of airport K at time t
A_k = {}

# Capacity of sector j at time t
S_j = {}


# Scheduled departure times
d_f = {F[0]: 1, 
       F[1]: 1, 
       F[2]: 17}

# Scheduled arrivals times
a_f = {F[0]: 15,
       F[1]: 13,
       F[2]: 23}

# Turnaround times
s_f = {F[0]: 5,
       F[1]: 5,
       F[2]: 5}

# Origin and Destination Airport
origin_f={}
origin_f[F[0]] = K[0]
origin_f[F[1]] = K[0]
origin_f[F[2]] = K[1]

destination_f={}
destination_f[F[0]]= K[2]
destination_f[F[1]]= K[1]
destination_f[F[2]]= K[2]

# Nodes for flight f
N_f = {}

# Arcs for flight f
arcs = {}

# Number of time units that flight f must spend in arc m,n
l_fnm = {}

# Scheduled time of arrival of flight f to arrive at node n (not considering possible delays)
r_fn = {}

# maximum ground holding time units delay per flight f
G = {}

# maximum airborne unit delays for flight f
A = {}

# Smallest time interval for flight f to arrive at node n
T_fn = {}

# Unit cost for air delay
ca = {}

# Unit cost for ground delay
cg = {}

# Nodes and arcs belonging to Sector j that are overflown by Flight f
# [Note that N_S denotes what in the statement is N_{jf}]
N_S = {}
arc_S = {}

# print('origin airport:', origin_f)
# print('dstination airport:', destination_f)
# print('Times', T)
# print('Nodes in Sector S', N_S)
# print('Arcs in Sector S', arc_S)
# print('Set of Times to overfly node n for flight f', T_fn)
# print('Nf', N_f)

In [117]:
############### Create a concrete model ###############
model = ConcreteModel('ATFM')

In [118]:
############### Initialize Sets and Parameters ###############
############### FEW examples are given below (complete until you get the print that you will find in the cell below) ###############

# pyright: reportAttributeAccessIssue=false 

model.T = Set(initialize=T, ordered=True)
model.F = Set(initialize=F, ordered=True)
model.K = Set(initialize=K, ordered=True)
model.S = Set(initialize=S, ordered=True)
model.C = Set(initialize=C, ordered=True)

model.G = Param(G.keys(), initialize=G_ini(G))
model.A = Param(A.keys(), initialize=A_ini(A))

model.ca = Param(ca.keys(), initialize=ca_ini(ca))
model.cg = Param(cg.keys(), initialize=cg_ini(cg))

model.N_S   = Set(N_S.keys(), initialize=N_S_ini(N_S), ordered=True)
model.arc_S = Set(arc_S.keys(), initialize=arc_S_ini(arc_S), ordered=True)
model.D_k = Param(D_k.keys(), initialize=D_k_ini(D_k, time_intervals))
model.A_k = Param(A_k.keys(), initialize=A_k_ini(A_k, time_intervals))
model.S_j = Param(S_j.keys(), initialize=S_j_ini(S_j, time_intervals))

model.d_f = Param(d_f.keys(), initialize=d_f)
model.a_f = Param(a_f.keys(), initialize=a_f)
model.s_f = Param(s_f.keys(), initialize=s_f)

model.origin_f = Param(model.F, initialize=origin_f)
model.destination_f = Param(model.F, initialize=destination_f)

model.N_f   = Set(N_f.keys(), initialize=N_f_ini(N_f), ordered=True)
model.arcs  = Set(arcs.keys(), initialize=arcs_ini(arcs), ordered=True)
model.l_fnm = Param(l_fnm.keys(), initialize=l_fnm_ini(l_fnm))
model.r_fn  = Param(r_fn.keys(), initialize=r_fn_ini(r_fn,model))
model.T_fn  = Set(T_fn.keys(), initialize=T_fn_ini(T_fn,model, time_intervals))


print('T_fn:', T_fn)
print('N_f:', N_f)

T_fn: {('f1', 'A'): array([1., 2., 3.]), ('f1', 'q(A)'): array([1., 2., 3.]), ('f1', 'a'): array([ 6.,  7.,  8.,  9., 10., 11., 12., 13.]), ('f1', 'b'): array([14., 15., 16., 17., 18., 19., 20., 21.]), ('f1', 'C'): array([15., 16., 17., 18., 19., 20., 21., 22.]), ('f2', 'A'): array([1., 2., 3., 4.]), ('f2', 'q(A)'): array([1., 2., 3., 4.]), ('f2', 'e'): array([ 6.,  7.,  8.,  9., 10., 11., 12., 13.]), ('f2', 'B'): array([13., 14., 15., 16., 17., 18., 19., 20.]), ('f3', 'B'): array([17., 18., 19., 20.]), ('f3', 'q(B)'): array([17., 18., 19., 20.]), ('f3', 'f'): array([20., 21., 22., 23., 24., 25., 26., 27.]), ('f3', 'C'): array([23., 24., 25., 26., 27., 28., 29., 30.])}
N_f: {'f1': ('A', 'q(A)', 'a', 'b', 'C'), 'f2': ('A', 'q(A)', 'e', 'B'), 'f3': ('B', 'q(B)', 'f', 'C')}


# You should get this print
T_fn: {('f1', 'A'): array([1., 2., 3.]), ('f1', 'q(A)'): array([1., 2., 3.]), ('f1', 'a'): array([ 6.,  7.,  8.,  9., 10., 11., 12., 13.]), ('f1', 'b'): array([14., 15., 16., 17., 18., 19., 20., 21.]), ('f1', 'C'): array([15., 16., 17., 18., 19., 20., 21., 22.]), ('f2', 'A'): array([1., 2., 3., 4.]), ('f2', 'q(A)'): array([1., 2., 3., 4.]), ('f2', 'e'): array([ 6.,  7.,  8.,  9., 10., 11., 12., 13.]), ('f2', 'B'): array([13., 14., 15., 16., 17., 18., 19., 20.]), ('f3', 'B'): array([17., 18., 19., 20.]), ('f3', 'q(B)'): array([17., 18., 19., 20.]), ('f3', 'f'): array([20., 21., 22., 23., 24., 25., 26., 27.]), ('f3', 'C'): array([23., 24., 25., 26., 27., 28., 29., 30.])}

N_f: {'f1': ('A', 'q(A)', 'a', 'b', 'C'), 'f2': ('A', 'q(A)', 'e', 'B'), 'f3': ('B', 'q(B)', 'f', 'C')}

In [119]:
############### Define Variables ###############
# Binary Variables

# pyright: reportAttributeAccessIssue=false 

var = {}
var["f1","A"] = ""
var["f1","q(A)"] = ""
var["f1","a"] = ""
var["f1","b"] = ""
var["f1","C"] = ""

var["f2","A"] = ""
var["f2","q(A)"] = ""
var["f2","e"] = ""
var["f2","B"] = ""

var["f3","B"] = ""
var["f3","q(B)"] = ""
var["f3","f"] = ""
var["f3","C"] = ""


# complete the definition of the variables until you get the print that you will find in the cell below
model.w = Var(var, model.T, within=Binary)

In [120]:
### Define Objective Function ###

# pyright: reportAttributeAccessIssue=false

model.objective_func = Objective(rule=Objective_rule, sense=minimize) 

In [121]:
################### Define Departure Capacity constraint ####################
# Constraint (1)
model.departure_capacity_constraint = Constraint(model.K, model.T, rule=departure_capacity_constraint)


In [122]:
################### Define Arrival Capacity constraint ####################
# Constraint (2)
model.arrival_capacity_constraint = Constraint(model.K, model.T, rule=arrival_capacity_constraint)

In [123]:
################### Define Sector Capacity constraint ####################
# Constraint (3)
# pyright: reportAttributeAccessIssue=false
# pyright: reportArgumentType=false
# pyright: reportIndexIssue=false

model.sector_capacity_constraint = Constraint(model.S, model.T, rule=sector_capacity_constraint)

# For debugging purposes
for index, value in enumerate(model.N_S["f1", "I"]):
    print(f"Index: {index}, Value: {value}")



Index: 0, Value: A
Index: 1, Value: q(A)
Index: 2, Value: a


# You should get this print
Index: 0, Value: A
Index: 1, Value: q(A)
Index: 2, Value: a

In [124]:
######## Connectivity constratins 1 ########
# Create a set that combines the indices you need for the constraint
# pyright: reportAttributeAccessIssue=false

model.T_fn_combined = Set(dimen=3, initialize=lambda model: ((f, n, int(t)) 
                                                             for f in model.F 
                                                             for n in model.N_f[f] 
                                                             for t in model.T_fn[f,n]))
display(model.T_fn_combined)      

model.flight_structure_constraints_1 = Constraint(model.T_fn_combined, rule=flight_structure_constraints_1_rule)

T_fn_combined : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     3 :    Any :   78 : {('f1', 'A', 1), ('f1', 'A', 2), ('f1', 'A', 3), ('f1', 'q(A)', 1), ('f1', 'q(A)', 2), ('f1', 'q(A)', 3), ('f1', 'a', 6), ('f1', 'a', 7), ('f1', 'a', 8), ('f1', 'a', 9), ('f1', 'a', 10), ('f1', 'a', 11), ('f1', 'a', 12), ('f1', 'a', 13), ('f1', 'b', 14), ('f1', 'b', 15), ('f1', 'b', 16), ('f1', 'b', 17), ('f1', 'b', 18), ('f1', 'b', 19), ('f1', 'b', 20), ('f1', 'b', 21), ('f1', 'C', 15), ('f1', 'C', 16), ('f1', 'C', 17), ('f1', 'C', 18), ('f1', 'C', 19), ('f1', 'C', 20), ('f1', 'C', 21), ('f1', 'C', 22), ('f2', 'A', 1), ('f2', 'A', 2), ('f2', 'A', 3), ('f2', 'A', 4), ('f2', 'q(A)', 1), ('f2', 'q(A)', 2), ('f2', 'q(A)', 3), ('f2', 'q(A)', 4), ('f2', 'e', 6), ('f2', 'e', 7), ('f2', 'e', 8), ('f2', 'e', 9), ('f2', 'e', 10), ('f2', 'e', 11), ('f2', 'e', 12), ('f2', 'e', 13), ('f2', 'B', 13), ('f2', 'B', 14), ('f2', 'B', 15), ('f2', 'B', 16), ('f2', 'B', 17), (


You should get this print:

T_fn_combined : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     3 :    Any :   78 : {('f1', 'A', 1.0), ('f1', 'A', 2.0), ('f1', 'A', 3.0), ('f1', 'q(A)', 1.0), ('f1', 'q(A)', 2.0), ('f1', 'q(A)', 3.0), ('f1', 'a', 6.0), ('f1', 'a', 7.0), ('f1', 'a', 8.0), ('f1', 'a', 9.0), ('f1', 'a', 10.0), ('f1', 'a', 11.0), ('f1', 'a', 12.0), ('f1', 'a', 13.0), ('f1', 'b', 14.0), ('f1', 'b', 15.0), ('f1', 'b', 16.0), ('f1', 'b', 17.0), ('f1', 'b', 18.0), ('f1', 'b', 19.0), ('f1', 'b', 20.0), ('f1', 'b', 21.0), ('f1', 'C', 15.0), ('f1', 'C', 16.0), ('f1', 'C', 17.0), ('f1', 'C', 18.0), ('f1', 'C', 19.0), ('f1', 'C', 20.0), ('f1', 'C', 21.0), ('f1', 'C', 22.0), ('f2', 'A', 1.0), ('f2', 'A', 2.0), ('f2', 'A', 3.0), ('f2', 'A', 4.0), ('f2', 'q(A)', 1.0), ('f2', 'q(A)', 2.0), ('f2', 'q(A)', 3.0), ('f2', 'q(A)', 4.0), ('f2', 'e', 6.0), ('f2', 'e', 7.0), ('f2', 'e', 8.0), ('f2', 'e', 9.0), ('f2', 'e', 10.0), ('f2', 'e', 11.0), ('f2', 'e', 12.0), ('f2', 'e', 13.0), ('f2', 'B', 13.0), ('f2', 'B', 14.0), ('f2', 'B', 15.0), ('f2', 'B', 16.0), ('f2', 'B', 17.0), ('f2', 'B', 18.0), ('f2', 'B', 19.0), ('f2', 'B', 20.0), ('f3', 'B', 17.0), ('f3', 'B', 18.0), ('f3', 'B', 19.0), ('f3', 'B', 20.0), ('f3', 'q(B)', 17.0), ('f3', 'q(B)', 18.0), ('f3', 'q(B)', 19.0), ('f3', 'q(B)', 20.0), ('f3', 'f', 20.0), ('f3', 'f', 21.0), ('f3', 'f', 22.0), ('f3', 'f', 23.0), ('f3', 'f', 24.0), ('f3', 'f', 25.0), ('f3', 'f', 26.0), ('f3', 'f', 27.0), ('f3', 'C', 23.0), ('f3', 'C', 24.0), ('f3', 'C', 25.0), ('f3', 'C', 26.0), ('f3', 'C', 27.0), ('f3', 'C', 28.0), ('f3', 'C', 29.0), ('f3', 'C', 30.0)}
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
1 q(A) a (q(A),a)
1 q(A) a (q(A),a)
1 q(A) a (q(A),a)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
1 q(A) e (q(A),e)
1 q(A) e (q(A),e)
1 q(A) e (q(A),e)
1 q(A) e (q(A),e)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
0 B q(B) (B,q(B))
0 B q(B) (B,q(B))
0 B q(B) (B,q(B))
0 B q(B) (B,q(B))
1 q(B) f (q(B),f)
1 q(B) f (q(B),f)
1 q(B) f (q(B),f)
1 q(B) f (q(B),f)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)

In [125]:
######## Connectivity constratins 2 ########
# pyright: reportAttributeAccessIssue=false
model.flight_structure_constraints_2 = Constraint(model.T_fn_combined, rule=flight_structure_constraints_2_rule)

In [126]:
######## Connectivity constratins 3 ########
# pyright: reportAttributeAccessIssue=false
model.flight_structure_constraints_3 = Constraint(model.T_fn_combined, rule=flight_structure_constraints_3_rule)

In [127]:
######## Connectivity constratins 4 ########
# pyright: reportAttributeAccessIssue=false
model.T_combined = Set(dimen=3, initialize=lambda model: ((f, n, int(t)) 
                                                             for f in model.F 
                                                             for n in model.N_f[f] 
                                                             for t in model.T))

model.flight_structure_constraints_4 = Constraint(model.T_combined, rule=flight_structure_constraints_4_rule)


In [128]:
######## Connectivity constratins 5 ########
# pyright: reportAttributeAccessIssue=false

# Create a set that combines the indices you need for the constraint
model.N_f_combined = Set(dimen=2, initialize=lambda model: ((f, n) for f in model.F for n in model.N_f[f]))
# display(model.N_f_combined)

model.flight_structure_constraints_5 = Constraint(model.N_f_combined, rule=flight_structure_constraints_5_rule)

In [129]:
# Create a set that combines the indices you need for the constraint
# pyright: reportAttributeAccessIssue=false
    
model.flight_structure_constraints_6 = Constraint(model.N_f_combined, model.T, rule=flight_structure_constraints_6_rule)
# model.flight_structure_constraints_6.pprint()

In [134]:
# Solver call
# pyright: reportAttributeAccessIssue=false

solver = SolverFactory('glpk')
#solver = SolverFactory('cplex_direct')
result = solver.solve(model)
# pyomo.environ.value(model.objective_func)

objective_val = model.objective_func()
print("Objective value after minimizing:", objective_val)

Objective value after minimizing: 3.0


In [131]:
# Prints: one can print the complete model or just some elements of it

# model.pprint()
#model.obj.pprint()
#model.departure_capacity_constraint.pprint()
#model.arrival_capacity_constraint.pprint()
#model.sector_capacity_constraint.pprint()
#model.flight_structure_constraints_1.pprint()
#model.Connectivity2.pprint()
#model.Connectivity3_bis.pprint()
#model.flight_structure_constraints_4.pprint()
#model.connectivity_constraint4_bis.pprint()
#model.flight_structure_constraints_5.pprint()
# model.w.pprint()
# model.objective_func.pprint()
# model.T_fn.pprint()
